<h1>Testing of the models starts here </h1>

In [102]:
testing_img = 'testing/T42RTN_20240321T060629_TCI_10m.jp2'

Breaking the image into (100x100)px smaller images 

In [103]:
import os
from PIL import Image

def split_image_to_fixed_size(image_path, tile_size, output_folder):
    """
    Splits an image into smaller images of a fixed size and saves them to a specified folder.

    Args:
        image_path (str): Path to the input image.
        tile_size (tuple): Size of each tile (width, height).
        output_folder (str): Folder to save the smaller images.

    Returns:
        None
    """
    # Open the input image
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Calculate the size of each tile
    tile_width, tile_height = tile_size

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Split the image into smaller images and save them
    for top in range(0, image_height, tile_height):
        for left in range(0, image_width, tile_width):
            right = min(left + tile_width, image_width)
            bottom = min(top + tile_height, image_height)

            # Crop the image
            cropped_image = image.crop((left, top, right, bottom))

            # Save the cropped image
            cropped_image.save(os.path.join(output_folder, f'image_{top}_{left}.png'))


In [104]:
split_image_to_fixed_size(testing_img, (100, 100), 'testing/input_images')

Check for objects and remove unnecessary

In [105]:
def process_and_filter_images(input_folder):
    """
    Processes images in the input folder to detect objects and deletes images with no objects.

    Args:
        input_folder (str): Path to the folder containing images.

    Returns:
        None
    """
    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)

        # Read image and preprocess
        image = cv2.imread(image_path)
        if image is None:
            print(f"Could not read image {image_name}. Skipping.")
            continue

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (9, 9), 0)

        edged = cv2.Canny(blur, 50, 100)
        edged = cv2.dilate(edged, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)

        # Find contours
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)

        if cnts:
            # Draw rectangles around objects
            for cnt in cnts:
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Convert image from BGR to RGB for displaying with Matplotlib
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            # If no contours are found, delete the image
            os.remove(image_path)
            print(f"No objects found in {image_name}. Image deleted")

process_and_filter_images('testing/input_images')


No objects found in image_2700_4600.png. Image deleted
No objects found in image_6900_8100.png. Image deleted
No objects found in image_9800_3500.png. Image deleted
No objects found in image_2900_400.png. Image deleted
No objects found in image_10800_3600.png. Image deleted
No objects found in image_10600_10600.png. Image deleted
No objects found in image_5300_2200.png. Image deleted
No objects found in image_8100_4900.png. Image deleted
No objects found in image_9800_7000.png. Image deleted
No objects found in image_10800_900.png. Image deleted
No objects found in image_5300_6700.png. Image deleted
No objects found in image_10800_7300.png. Image deleted
No objects found in image_8500_4000.png. Image deleted
No objects found in image_7200_10500.png. Image deleted
No objects found in image_2000_2000.png. Image deleted
No objects found in image_5400_4400.png. Image deleted
No objects found in image_8200_6300.png. Image deleted
No objects found in image_2400_2900.png. Image deleted
No obj

Move all land images to another folder

In [106]:
def predictions(model, img_path,IMG_SIZE):
    image = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    preds = model.predict(image)
    i = np.argmax(preds[0])
    return i

In [107]:
land_model=load_model('land_model.h5')

In [108]:
import os
import shutil

def filter_and_move_files(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through files in input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        preds = predictions(land_model,file_path,100)
        # Check if the file meets the condition
        if preds==0:
            # Move the file to the output folder
            shutil.move(file_path, os.path.join(output_folder, filename))
            print(f"File {file_path} has been moved to the folder")


In [109]:
input_folder = 'testing/input_images'
output_folder = 'testing/land_images'

# Filter and move files based on the condition
filter_and_move_files(input_folder, output_folder)

1/1 [==============================] - 0s 20ms/step
File testing/input_images/image_1900_10900.png has been moved to the folder
1/1 [==============================] - 0s 18ms/step
File testing/input_images/image_4900_7400.png has been moved to the folder
1/1 [==============================] - 0s 18ms/step
File testing/input_images/image_4400_9200.png has been moved to the folder
1/1 [==============================] - 0s 17ms/step
File testing/input_images/image_400_10100.png has been moved to the folder
1/1 [==============================] - 0s 17ms/step
File testing/input_images/image_6100_10400.png has been moved to the folder
1/1 [==============================] - 0s 17ms/step
File testing/input_images/image_3700_9000.png has been moved to the folder
1/1 [==============================] - 0s 17ms/step
File testing/input_images/image_3300_9900.png has been moved to the folder
1/1 [==============================] - 0s 16ms/step
File testing/input_images/image_600_10900.png has been mo

The code below is for looking at all the images that have ships

In [110]:
import os
import shutil

def filter_and_move_files(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through files in input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        preds = predictions(model,file_path,48)
        # Check if the file meets the condition
        if preds==1:
            # Move the file to the output folder
            shutil.move(file_path, os.path.join(output_folder, filename))
            print(f"File {file_path} has been moved to the folder")


In [ ]:
input_folder = 'testing/input_images'
output_folder = 'testing/ship_images'

# Filter and move files based on the condition
filter_and_move_files(input_folder, output_folder)

It is observed that the false positive rate for ship classification is quite high due to the presence of clouds. To counter this, another model for removing the images that have clouds can be trained, and shall be done subsequently. 